# EXPONENTIATION AND HAMILTONIAN SIMULATION

This note demonstrates how to use the Classiq platform exponentiation module to solve Hamiltonian simulation problems. This demonstrates the strength of the Classiq exponentiation module.

## CHEMICAL SIMULATION

Chemical simulation is one of the most exciting applications for quantum computers. When precise simulations of electron-electron interactions are necessary, it is sometimes possible to use a classical computer, but classical computers struggle to simulate more complex molecular interactions. It is best to simulate these particle interactions at the quantum level, and an excellent way to do this is with a quantum computer.

The ability to accurately simulate molecular interactions will have extensive applications. When used for drug discovery, it will allow for the rapid development of vaccines and new cures for diseases. In materials research, we can hope to discover materials with higher strength-to-weight ratios and environmentally-friendly building materials.

## THE H<sub>2</sub>O HAMILTONIAN SIMULATION PROBLEM

We show how to generate a circuit that approximates the unitary $e^{-iH}$ where $H$ is the qubit hamiltonian of a H<sub>2</sub>O (water) molecule. The H<sub>2</sub>O Hamiltonian is composed of 551 Pauli strings on 8 qubits.

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.applications.chemistry import Molecule, MoleculeProblem
from classiq.builtin_functions import SuzukiTrotter
from classiq.builtin_functions.suzuki_trotter import SuzukiParameters
from classiq.model import CustomHardwareSettings, Preferences

molecule_H2O = Molecule(
    atoms=[("O", (0.0, 0.0, 0.0)), ("H", (0, 0.586, 0.757)), ("H", (0, 0.586, -0.757))]
)

gs_problem = MoleculeProblem(
    molecule=molecule_H2O,
    basis="sto3g",
    mapping="jordan_wigner",
    z2_symmetries=False,
    freeze_core=True,
)

hamiltonian = gs_problem.generate_hamiltonian()

trotter_params = SuzukiTrotter(
    pauli_operator=hamiltonian,
    evolution_coefficient=1,
    suzuki_parameters=SuzukiParameters(order=1, repetitions=1),
    use_naive_evolution=False,
)

model = Model(
    preferences=Preferences(
        custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"])
    )
)
model.SuzukiTrotter(params=trotter_params)
qmod = model.get_model()

with open("exponentiation.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

print(f"Classiq's exponentiation depth is {circuit.transpiled_circuit.depth}")
print(
    f"Classiq's exponentiation CX-count is {circuit.transpiled_circuit.count_ops['cx']}"
)
circuit.show()

These impressive results can be compared to the naive exponentiation modules often found in the literature:

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.applications.chemistry import Molecule, MoleculeProblem
from classiq.builtin_functions import SuzukiTrotter
from classiq.builtin_functions.suzuki_trotter import SuzukiParameters
from classiq.model import CustomHardwareSettings, Preferences

molecule_H2O = Molecule(
    atoms=[("O", (0.0, 0.0, 0.0)), ("H", (0, 0.586, 0.757)), ("H", (0, 0.586, -0.757))]
)

gs_problem = MoleculeProblem(
    molecule=molecule_H2O,
    basis="sto3g",
    mapping="jordan_wigner",
    z2_symmetries=False,
    freeze_core=True,
)

hamiltonian = gs_problem.generate_hamiltonian()

trotter_params = SuzukiTrotter(
    pauli_operator=hamiltonian,
    evolution_coefficient=1,
    suzuki_parameters=SuzukiParameters(order=1, repetitions=1),
    use_naive_evolution=True,
)

model = Model(
    preferences=Preferences(
        custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"])
    )
)
model.SuzukiTrotter(params=trotter_params)
qmod = model.get_model()

with open("exponentiation_h2o_naive.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(qmod)
circuit = GeneratedCircuit.from_qprog(qprog)

print(f"Naive exponentiation depth is {circuit.transpiled_circuit.depth}")
print(f"Naive exponentiation CX-count is {circuit.transpiled_circuit.count_ops['cx']}")
circuit.show()

One can clearly see the dramatic decrease in both depth and number of CX-gates.

## AUTOMATIC ERROR REDUCTION

The Classiq exponentiation module provides error management and automatically minimizes the error and determines the best Trotter-Suzuki order and repetitions for any provided depth. We demonstrate this with an arbitrary inputted Pauli operator on 8 qubits.

In [ ]:
from classiq import GeneratedCircuit, Model, synthesize
from classiq.builtin_functions import Exponentiation
from classiq.builtin_functions.exponentiation import (
    ExponentiationConstraints,
    PauliOperator,
)

pauli_operator = PauliOperator(
    pauli_list=[
        ("IIZXXXII", 0.1),
        ("IIXXYYII", 0.2),
        ("IIIIZZYX", 0.3),
        ("XZIIIIIX", 0.4),
        ("IIIIIZXI", 0.5),
        ("IIIIIIZY", 0.6),
        ("IIIIIIXY", 0.7),
        ("IIYXYZII", 0.8),
        ("IIIIIIXZ", 0.9),
        ("IIYZYIII", 1.0),
    ]
)
exponentiation_params = Exponentiation(
    pauli_operator=pauli_operator,
    evolution_coefficient=0.05,
    constraints=ExponentiationConstraints(
        max_depth=400,
    ),
    optimization="MINIMIZE_ERROR",
)

model = Model()
model.Exponentiation(params=exponentiation_params)
qmod = model.get_model()

with open("exponentiation_minimize_error.qmod", "w") as f:
    f.write(qmod)

qprog = synthesize(model.get_model())
circuit = GeneratedCircuit.from_qprog(qprog)

circuit.show()

The Classiq engine automatically opted for 6 second-order Suzuki-Trotter layers instead of 12 first-order layers in order to minimize the error of the exponentiation within the depth constraints.

## CONCLUSION

Classiq has packaged the domain expertise of dozens of its scientists and quantum software engineers into the software platform. The result: a system that can automatically generate efficient quantum circuits for complex problems, making it faster and easier than ever to solve real-life problems with quantum computing. When the circuits are of manageable size, Classiq creates solutions that are on par with the best manually-created circuits. When the circuits are larger than those a human can reasonably create, Classiq allows you to progress farther because of its powerful capabilities.
